In [1]:
#import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pp
from vdom import pre
from xgboost import plot_importance

In [2]:
# import Models and datasets
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
train = pd.read_csv('C:/Users/Hp/Desktop/titanic2019/train.csv') #Train dataset
test = pd.read_csv('C:/Users/Hp/Desktop/titanic2019/test.csv') #Test dataset
fullset = train.append(test, ignore_index = True, sort=True)

In [3]:
pp.ProfileReport(fullset)

Number of variables,12
Number of observations,1309
Total Missing (%),10.8%
Total size in memory,122.8 KiB
Average record size in memory,96.1 B
Numeric,7
Categorical,5
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [4]:
pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
display(pre("Note: This data is sampling"))
fullset.head()

<pre>Note: This data is sampling</pre>

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [5]:
# data cleaning and attribute engineering
fullset["Sex"].replace(["male","female"],[0,1],inplace=True)
fullset["Age"].fillna(fullset["Age"].median(), inplace = True)
fullset["Fare"].fillna(fullset["Fare"].median(), inplace = True)
fullset['Title'] = fullset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
fullset['Title'] = fullset['Title'].replace(['Dona','Sir','Dr','Don','Dona','Lady','Countess','Capt','Col','Major','Rev','Jonkheer'],'Other')
title_set = {"Mr":0, "Mrs":1, "Miss":2, "Master":3, "Other":4}
fullset['Title'].replace(['Mlle','Ms','Mme'],['Miss','Miss','Mrs'],inplace=True)
fullset['Title'] = fullset['Title'].map(title_set)
fullset['Title'].fillna(0, inplace = True)
fullset["Embarked"].replace(["S","C","Q"],[0,1,2],inplace = True)
fullset["FamilySize"] = fullset["SibSp"] + fullset["Parch"] + 1
train = fullset[:891]
test = fullset[891:]
print("Done.")

Done.


In [6]:
# create the final attribute sets
tree_features_X = train[["Age","Sex","Pclass","Fare","Title","Embarked","FamilySize"]].values
test_X = test[["Age","Sex","Pclass","Fare","Title","Embarked","FamilySize"]].values
tree_features_Y = train["Survived"].values
train_X,valid_X,train_Y,valid_Y = train_test_split(tree_features_X, tree_features_Y, train_size = 0.66)
print("Done.")

Done.


In [ ]:
# create the model and compare accuracy
rf = RandomForestClassifier(max_depth=3, n_estimators=100000, criterion="entropy", random_state=10)
rf.fit(train_X, train_Y)
print("Model accuracy : ",rf.score(train_X, train_Y))
print("Valid accuracy : ",rf.score(valid_X, valid_Y))
print("Attr importance: ",rf.feature_importances_)

In [ ]:
scores = cross_val_score(rf, train_X, train_Y, cv=10, scoring = "accuracy")
print(scores)

In [ ]:
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

In [ ]:
# save results output to Kaggle-required csv format
modelPrediction = randomForest.predict(test_X).astype(int)
passId = fullset[891:].PassengerId.astype(int)
output = pd.DataFrame( { 'PassengerId': passId, 'Survived': modelPrediction } )
output.to_csv( 'New_Abel_Prediction3.csv' , index = False )

print (modelPrediction)
print('Done.')

# done.

In [ ]:
from xgboost import XGBRegressor

xgc_model = XGBRegressor(objective = 'reg:linear', colsample_bytree =0.3,learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimator=1000)

In [ ]:
# Add silent=True to avoid printing out updates with each cycle
xgb_train = xgc_model.fit(train_X, train_Y)
new_predictions = xgc_model.predict(test_X).astype(int)
print(new_predictions)

In [ ]:
Latest_prediction = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": new_predictions
    })
Latest_prediction.to_csv('Latest_prediction.csv', index=False)

In [ ]:
acc_xgb = round(xgb_train.score(x_train, train_Y) * 100, 2)
print(round(acc_xgb,2,), "%")